# Generate test data

In [3]:
import numpy as np
import pandas as pd
import paramiko
import re
import copy

###### Load markable file

In [3]:
markables = pd.read_csv('markable2.csv',sep=',')

In [4]:
markables = markables.drop(columns=['Unnamed: 0'])

In [5]:
markables

,doc_id,sentence,word_number,word,tagging,?,lemma,fourth*,FEATS,head,gramFct,co_num
0,T890221.159,67714.0,1.0,Amnesty,NE,(VROOT:--(SIMPX:--(VF:-(NX=ORG:ON*)),Amnesty,(ORG),Case=Nom|Number=Sing,2,nsubj,9994
1,T890221.159,67714.0,3.0,DDR-Geheimjustiz,NN,(MF:-(NX:OA*)))),DDR-Geheimjustiz,*,Case=Acc|Gender=Fem|Number=Sing,2,obj,9995
2,T890221.159,67714.0,1.0,Amnesty,NE,(VROOT:--(SIMPX:--(VF:-(NX=ORG:ON*)),Amnesty,(ORG),Case=Nom|Number=Sing,2,nsubj,0
3,T890221.159,67715.0,1.0,Der,ART,(VROOT:--(SIMPX:--(VF:-(NX:ON(NX:HD(NX:HD*,der,*,Case=Nom|Definite=Def|Gender=Masc|Number=Sing|...,2,det,9996
4,T890221.159,67715.0,2.0,Bericht,NN,*),Bericht,*,Case=Nom|Gender=Masc|Number=Sing,20,nsubj:pass,9996
5,T890221.159,67715.0,4.0,amnesty,NE,(NX=ORG:HD*,Amnesty,(ORG*,Case=Dat|Number=Sing,2,nmod,9997
6,T890221.159,67715.0,5.0,international,NE,*))),International,*),Case=Dat|Number=Sing,4,flat,9997
7,T890221.159,67715.0,7.0,Menschenrechtsverletzungen,NN,(NX:HD(NX:HD*),Menschenrechtsverletzung,*,Case=Acc|Gender=Fem|Number=Plur,2,nmod,9998
8,T890221.159,67715.0,9.0,der,ART,(NX=GPE:HD*,die,*,Case=Dat|Definite=Def|Gender=Fem|Number=Sing|P...,12,det,9999
9,T890221.159,67715.0,10.0,Deutschen,ADJA,(ADJX:-*),deutsch,(GPE*,Case=Dat|Gender=Fem|Number=Sing,12,amod,9999


###### Load test_coref file

In [7]:
df_corf = pd.read_csv('test_df2_coref.csv',sep = ',')

In [8]:
#remove columns that not going to be used
df_corf = df_corf.drop(columns=['Unnamed: 0'])
df_corf = df_corf.drop(columns=['first-','second-','third-','fifth-'])
#drop rows with NaN data
df_corf = df_corf.dropna(axis = 0 , how = 'all')
df_corf = df_corf.reset_index(drop = True)

In [9]:
df_corf

,doc_id,sentence,word_number,word,tagging,?,lemma,fourth*,coreference_chain_id
0,T890221.159,67714.0,1.0,Amnesty,NE,(VROOT:--(SIMPX:--(VF:-(NX=ORG:ON*)),Amnesty,(ORG),(0)
1,T890221.159,67714.0,2.0,kritisiert,VVFIN,(LK:-(VXFIN:HD*)),kritisieren,*,-
2,T890221.159,67714.0,3.0,DDR-Geheimjustiz,NN,(MF:-(NX:OA*)))),DDR-Geheimjustiz,*,-
3,T890221.159,67715.0,1.0,Der,ART,(VROOT:--(SIMPX:--(VF:-(NX:ON(NX:HD(NX:HD*,der,*,-
4,T890221.159,67715.0,2.0,Bericht,NN,*),Bericht,*,-
5,T890221.159,67715.0,3.0,von,APPR,(PX:-*,von,*,-
6,T890221.159,67715.0,4.0,amnesty,NE,(NX=ORG:HD*,Amnesty,(ORG*,(0
7,T890221.159,67715.0,5.0,international,NE,*))),International,*),0)
8,T890221.159,67715.0,6.0,über,APPR,(PX:-*,über,*,-
9,T890221.159,67715.0,7.0,Menschenrechtsverletzungen,NN,(NX:HD(NX:HD*),Menschenrechtsverletzung,*,-


###### Load test_sync file

In [8]:
df_sync_2 = pd.read_csv("test_df_sync_2.csv",sep=',')
df_sync_2 = df_sync_2.drop(columns=['Unnamed: 0'])

In [21]:
df_sync_2

,doc_id,word_number,word,lemma,POS,FEATS,head,gramFct,?????
0,T890221.159,1,Amnesty,Amnesty,NE,Case=Nom|Number=Sing,2,nsubj,NE=ORG_1209866|Morph=ns*|TopoField=VF
1,T890221.159,2,kritisiert,kritisieren,VVFIN,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...,0,root,Morph=3sis|TopoField=LK
2,T890221.159,3,DDR-Geheimjustiz,DDR-Geheimjustiz,NN,Case=Acc|Gender=Fem|Number=Sing,2,obj,Morph=asf|TopoField=MF
3,T890221.159,1,Der,der,ART,Case=Nom|Definite=Def|Gender=Masc|Number=Sing|...,2,det,Morph=nsm|TopoField=VF
4,T890221.159,2,Bericht,Bericht,NN,Case=Nom|Gender=Masc|Number=Sing,20,nsubj:pass,Morph=nsm|TopoField=VF
5,T890221.159,3,von,von,APPR,Case=Dat,4,case,Morph=d|TopoField=VF
6,T890221.159,4,amnesty,Amnesty,NE,Case=Dat|Number=Sing,2,nmod,Corr=Amnesty|NE=ORG_1209872|Morph=ds*|TopoFiel...
7,T890221.159,5,international,International,NE,Case=Dat|Number=Sing,4,flat,Corr=International|NE=ORG_1209872|Morph=ds*|To...
8,T890221.159,6,über,über,APPR,Case=Acc,7,case,Morph=a|TopoField=VF
9,T890221.159,7,Menschenrechtsverletzungen,Menschenrechtsverletzung,NN,Case=Acc|Gender=Fem|Number=Plur,2,nmod,Morph=apf|TopoField=VF


###### Class for Stack and Node

In [9]:
class Node:  
    def __init__(self, value):
        self.value = value
        self.next = None
  
  
class Stack: 
    def __init__(self):
        self.top = None
  
    def push(self, value):
        node = Node(value)
        node.next = self.top
        self.top = node
  
    def pop(self):
        node = self.top
        if node is None:
            raise Exception('This is an empty stack')
        self.top = node.next
        return node.value
  
    def peek(self):
        node = self.top
        if node is None:
            raise Exception('This is an empty stack')
        return node.value
  
    def is_empty(self):
        return not self.top
  
    def size(self):
        node = self.top
        count = 0
        if node is None:
            raise Exception('This is an empty stack')
        while node is not None:
            count += 1
            node = node.next
        return count

###### Group by based on doc_id for test_coref file
Calculate how many words in one segment

In [10]:
groupby_doc_id = df_corf.groupby('doc_id').size()

In [11]:
groupby_doc_id = groupby_doc_id.reset_index()

In [12]:
groupby_doc_id.columns = ['doc_id','total_counts']

In [25]:
groupby_doc_id

,doc_id,total_counts
0,T890221.159,1083
1,T890222.4,2110
2,T890223.132,966
3,T890224.87,1038
4,T890225.29,459
5,T890225.91,1222
6,T890227.28,1010
7,T890227.29,1074
8,T890227.44,801
9,T890227.71,1362


###### Extract true label from test_coref file
First : extract mentions

In [ ]:
stack = Stack()
stack_temp = Stack()
data_len = len(df_corf)
for i in range(0, data_len):
    sentid = df_corf.iloc[i]['sentence']
    tokenid = df_corf.iloc[i]['word_number']
    seg = df_corf.iloc[i]['doc_id']
    if df_corf.iloc[i]['coreference_chain_id'].find('|'):
        ids = df_corf.iloc[i]['coreference_chain_id'].split('|')
    else:
        ids = df_corf.iloc[i]['coreference_chain_id']
    for identry in ids:
        co_id_list = re.findall(r'\d+', identry)
        if len(co_id_list):
            co_id = co_id_list[0]
        if identry.startswith('(') and identry.endswith(')'):
            stack.push([co_id,seg,sentid,tokenid,tokenid])
        elif identry.startswith('('):
                stack_temp.push([co_id,seg,sentid,tokenid])
        elif identry.endswith(')'):
                lastentry = stack_temp.pop()
                lastentry.append(tokenid)
                stack.push(lastentry)

In [14]:
#copy stack generated last step for further use
stack_test = Stack()
stack_test = copy.copy(stack)
stack_test_2 = copy.copy(stack)
stack_test_3 = copy.copy(stack)

In [15]:
#Pop mentions from the stack and store them in a dataframe
chain_true_label = pd.DataFrame()
for i in range(0, stack.size()):
    top  = stack_test.pop()
    chain_true_label = chain_true_label.append([top])

In [16]:
#reset index and rename columns
chain_true_label= chain_true_label.reset_index(drop=True)
chain_true_label.columns = ['core_id', 'doc_id', 'sent_num', 'phrase_start','phrase_end']

In [17]:
chain_true_label

,core_id,doc_id,sent_num,phrase_start,phrase_end
0,58,T890413.102,72643.0,18.0,18.0
1,58,T890413.102,72643.0,10.0,10.0
2,4,T890413.102,72643.0,7.0,7.0
3,57,T890413.102,72643.0,1.0,1.0
4,57,T890413.102,72642.0,8.0,9.0
5,4,T890413.102,72642.0,7.0,7.0
6,4,T890413.102,72642.0,2.0,2.0
7,4,T890413.102,72641.0,5.0,5.0
8,4,T890413.102,72640.0,10.0,10.0
9,4,T890413.102,72640.0,7.0,7.0


In [18]:
#Calculate how many mentions in one segment
groupby_ture_chain = chain_true_label.groupby('doc_id').size()
groupby_ture_chain = groupby_ture_chain.reset_index()
groupby_ture_chain.columns = ['doc_id','total_counts']

In [27]:
groupby_ture_chain

,doc_id,total_counts
0,T890221.159,65
1,T890222.4,270
2,T890223.132,78
3,T890224.87,111
4,T890225.29,60
5,T890225.91,175
6,T890227.28,103
7,T890227.29,100
8,T890227.44,69
9,T890227.71,86


In [19]:
groupby_ture_chain_2 = chain_true_label.groupby('doc_id').count().reset_index()

In [20]:
#Calculate how many mentions in one segment with the same coref_id
groupby_ture_chain_core_id = chain_true_label.groupby(['core_id','doc_id']).count().reset_index()

In [21]:
#Merge the dataframe for all mentions with the count for all coref_id(the dataframe generate last step)
true_chain_merge_count = chain_true_label.merge(groupby_ture_chain_core_id,on=['core_id','doc_id'],how='outer')
true_chain_merge_count.drop(columns=['phrase_end_y'],inplace=True)
true_chain_merge_count.drop(columns=['sent_num_y'],inplace=True)
true_chain_merge_count = true_chain_merge_count.rename(columns={'sent_num_x':'sent_num','phrase_start_x':'phrase_start','phrase_end_x':'phrase_end','phrase_start_y':'counts'})

In [22]:
#Merge the dataframe from last step and the dataframe with count for mentions in each segment information in it.
#The new dataframe contains informations about count for a single coref_id and count for all mentions in one segment.
true_chain_merge_count_2 = true_chain_merge_count.merge(groupby_ture_chain,on=['doc_id'],how='outer')

In [46]:
true_chain_merge_count_2

,core_id,doc_id,sent_num,phrase_start,phrase_end,counts,total_counts
0,58,T890413.102,72643.0,18.0,18.0,2,283
1,58,T890413.102,72643.0,10.0,10.0,2,283
2,4,T890413.102,72643.0,7.0,7.0,24,283
3,4,T890413.102,72642.0,7.0,7.0,24,283
4,4,T890413.102,72642.0,2.0,2.0,24,283
5,4,T890413.102,72641.0,5.0,5.0,24,283
6,4,T890413.102,72640.0,10.0,10.0,24,283
7,4,T890413.102,72640.0,7.0,7.0,24,283
8,4,T890413.102,72640.0,5.0,5.0,24,283
9,4,T890413.102,72639.0,3.0,4.0,24,283


Second: generate goldchian use the dataframe generate last step and the algorithm from lecture 5

In [23]:
gold_chain_dict_with_core_id = dict() 
i = 0
while i < len(true_chain_merge_count_2):
    k = 0
    gold_chain_dict_with_core_id[true_chain_merge_count_2.iloc[i]['doc_id']] = [[(true_chain_merge_count_2.iloc[i]['core_id'],str(true_chain_merge_count_2.iloc[i]['sent_num']),str(true_chain_merge_count_2.iloc[i]['phrase_start']),str(true_chain_merge_count_2.iloc[i]['phrase_end']))]]
    j=0
    for j in range(true_chain_merge_count_2.iloc[i]['total_counts']):
        if i+1 < len(true_chain_merge_count_2):
            #if 2 mentions are in the same segment
            if true_chain_merge_count_2.iloc[i+1]['doc_id'] == true_chain_merge_count_2.iloc[i]['doc_id']:
                #if 2 mentions have same coref_id
                if true_chain_merge_count_2.iloc[i+1]['core_id'] == true_chain_merge_count_2.iloc[i]['core_id']:
                    gold_chain_dict_with_core_id[true_chain_merge_count_2.iloc[i]['doc_id']][k].append((true_chain_merge_count_2.iloc[i+1]['core_id'],str(true_chain_merge_count_2.iloc[i+1]['sent_num']),str(true_chain_merge_count_2.iloc[i+1]['phrase_start']),str(true_chain_merge_count_2.iloc[i+1]['phrase_end'])))
                #if they do not have same coref_id then create a new list for the new coref_id
                elif true_chain_merge_count_2.iloc[i+1]['core_id'] != true_chain_merge_count_2.iloc[i]['core_id']:
                    k = k+1
                    gold_chain_dict_with_core_id[true_chain_merge_count_2.iloc[i]['doc_id']].append([(true_chain_merge_count_2.iloc[i+1]['core_id'],str(true_chain_merge_count_2.iloc[i+1]['sent_num']),str(true_chain_merge_count_2.iloc[i+1]['phrase_start']),str(true_chain_merge_count_2.iloc[i+1]['phrase_end']))])
            i = i+1
    if i+1 >= len(true_chain_merge_count_2):
         break

In [26]:
gold_chain_dict_with_core_id

{'T890221.159': [[('1', '67760.0', '41.0', '42.0'),
   ('1', '67749.0', '34.0', '35.0'),
   ('1', '67738.0', '37.0', '39.0'),
   ('1', '67738.0', '15.0', '16.0'),
   ('1', '67736.0', '39.0', '40.0'),
   ('1', '67734.0', '29.0', '30.0'),
   ('1', '67726.0', '17.0', '18.0'),
   ('1', '67725.0', '16.0', '17.0'),
   ('1', '67724.0', '3.0', '3.0'),
   ('1', '67723.0', '14.0', '15.0'),
   ('1', '67722.0', '26.0', '26.0'),
   ('1', '67721.0', '23.0', '24.0'),
   ('1', '67719.0', '31.0', '32.0'),
   ('1', '67716.0', '12.0', '12.0'),
   ('1', '67716.0', '8.0', '9.0'),
   ('1', '67715.0', '15.0', '17.0'),
   ('1', '67715.0', '9.0', '12.0')],
  [('15', '67760.0', '33.0', '34.0'), ('15', '67760.0', '11.0', '17.0')],
  [('14', '67758.0', '17.0', '17.0'), ('14', '67758.0', '14.0', '15.0')],
  [('13', '67756.0', '12.0', '13.0'), ('13', '67734.0', '41.0', '42.0')],
  [('12', '67751.0', '16.0', '16.0'),
   ('12', '67751.0', '14.0', '14.0'),
   ('12', '67751.0', '7.0', '7.0'),
   ('12', '67751.0', '1.0'

Third: use the goldchain generated last step to generate a new dataframe with mentions and features in it

In [ ]:
gold_list_with_features = list()
i=0
for seg in groupby_doc_id['doc_id']:
    segchain = gold_chain_dict_with_core_id.get(seg)
    
    for single_chain in segchain:
        for j in range(len(single_chain)):
            doc_id = seg
            core_id = single_chain[j][0] 
            sent_id = single_chain[j][1]
            start_token = single_chain[j][2]
            end_token = single_chain[j][3]
            index = df_corf[(df_corf['doc_id'] == doc_id) & (df_corf['sentence'] == float(sent_id)) & (df_corf['word_number'] == float(start_token))].index.tolist()
            POS = ""
            gramFct = ""
            FEATS = ""
            head_num = float("inf")
            if float(start_token) != float(end_token) :
                dis = float(end_token) - float(start_token)+1
                for k in range(int(dis)):
                    head = int(df_sync_2.iloc[index[0]+k]['head'])
                    if head <= head_num:
                        head_num = head
                        POS = df_sync_2.iloc[index[0]+k]['POS']
                        gramFct = df_sync_2.iloc[index[0]+k]['gramFct']
                        FEATS = df_sync_2.iloc[index[0]+k]['FEATS']
            else:
                POS = df_sync_2.iloc[index[0]]['POS']
                gramFct = df_sync_2.iloc[index[0]]['gramFct']
                FEATS = df_sync_2.iloc[index[0]]['FEATS']
                head_num = int(df_sync_2.iloc[index[0]]['head'])
            gold_list_with_features.append([core_id,seg,sent_id,start_token,end_token,gramFct,POS,head_num,FEATS])
            i=i+1

In [25]:
df_golden_list = pd.DataFrame(gold_list_with_features, columns=['core_id','seg','sent_id','start_token','end_token','gramFct','POS','head','FEATS']) 

In [58]:
df_golden_list

,core_id,seg,sent_id,start_token,end_token,gramFct,POS,head,FEATS
0,1,T890221.159,67760.0,41.0,42.0,det,ART,42,Case=Acc|Definite=Def|Gender=Neut|Number=Sing|...
1,1,T890221.159,67749.0,34.0,35.0,nmod:poss,NE,33,Case=Gen|Gender=Fem|Number=Sing
2,1,T890221.159,67738.0,37.0,39.0,amod,ADJA,39,Case=Acc|Gender=Neut|Number=Sing
3,1,T890221.159,67738.0,15.0,16.0,det,ART,16,Case=Dat|Definite=Def|Gender=Fem|Number=Sing|P...
4,1,T890221.159,67736.0,39.0,40.0,det,ART,40,Case=Dat|Definite=Def|Gender=Fem|Number=Sing|P...
5,1,T890221.159,67734.0,29.0,30.0,nmod,NE,27,Case=Gen|Gender=Fem|Number=Sing
6,1,T890221.159,67726.0,17.0,18.0,det,ART,18,Case=Nom|Definite=Def|Gender=Fem|Number=Sing|P...
7,1,T890221.159,67725.0,16.0,17.0,obl,NE,13,Case=Dat|Gender=Fem|Number=Sing
8,1,T890221.159,67724.0,3.0,3.0,nsubj,PPER,2,Case=Nom|Gender=Fem|Number=Sing|PronType=Prs
9,1,T890221.159,67723.0,14.0,15.0,nsubj,NE,13,Case=Nom|Gender=Fem|Number=Sing


###### Generate test data

First: calculate how many markables and how many words in a markabke in the markables file

In [29]:
mark_groupby_co_mun = markables.groupby('co_num').size()

In [30]:
mark_groupby_co_mun = mark_groupby_co_mun.reset_index()
mark_groupby_co_mun.columns = ['co_num','total_counts']

In [31]:
mark_groupby_co_mun

,co_num,total_counts
0,0,1
1,1,2
2,3,3
3,6,1
4,7,2
5,9,1
6,10,2
7,12,1
8,13,1
9,14,1


Second: generate a dataframe for markables with segment_id ,sent_id , start_token and end token

In [ ]:
    chain = pd.DataFrame(columns = ('doc_id','sentence','strat_token','end_token'))
    for j in range(len(mark_groupby_co_mun)):
        co_num = mark_groupby_co_mun.iloc[j]['co_num']
        total_counts = mark_groupby_co_mun.iloc[j]['total_counts']
        if total_counts == 1:
            markable = markables.loc[markables['co_num']==co_num].iloc[0]
            chain = chain.append(pd.DataFrame({'doc_id':[markable['doc_id']],'sentence':[markable['sentence']],'strat_token':[markable['word_number']],'end_token':[markable['word_number']]}),sort=False)
        elif total_counts>1:
            markable = markables.loc[markables['co_num']==co_num]
            length = len(markable)
            markable_start = markable.iloc[0]
            markable_end = markable.iloc[length-1]
            chain = chain.append(pd.DataFrame({'doc_id':[markable_start['doc_id']],'sentence':[markable_start['sentence']],'strat_token':[markable_start['word_number']],'end_token':[markable_end['word_number']]}),sort=False)
 

In [35]:
chain = chain.reset_index(drop=True)

In [36]:
#Group by based on segment_id
#Calculate how many markables in one segment
group_co_num = chain.groupby('doc_id').size()
group_co_num = group_co_num.reset_index()
group_co_num.columns = ['doc_id','total_counts']

In [38]:
group_co_num

,doc_id,total_counts
0,T890221.159,368
1,T890222.4,713
2,T890223.132,324
3,T890224.87,354
4,T890225.29,156
5,T890225.91,468
6,T890227.28,343
7,T890227.29,351
8,T890227.44,259
9,T890227.71,438


In [39]:
#Sort the chain based on ascending value of segment_id and sent_id
chain_sort = chain.sort_values(by=['doc_id','sentence'],ascending = (True,True)).reset_index(drop=True)

In [40]:
chain_sort

,doc_id,sentence,strat_token,end_token
0,T890221.159,67714.0,1.0,1.0
1,T890221.159,67714.0,1.0,1.0
2,T890221.159,67714.0,3.0,3.0
3,T890221.159,67715.0,4.0,5.0
4,T890221.159,67715.0,10.0,12.0
5,T890221.159,67715.0,1.0,2.0
6,T890221.159,67715.0,4.0,5.0
7,T890221.159,67715.0,7.0,7.0
8,T890221.159,67715.0,9.0,10.0
9,T890221.159,67715.0,12.0,12.0


In [41]:
#Calculate how many words in one sentences in each segment in test_coref file
sentence_num = df_corf.groupby(['doc_id','sentence']).count().reset_index()

In [42]:
sentence_num.drop(columns=['word','tagging','?','coreference_chain_id','lemma','fourth*'],inplace=True)

In [43]:
sentence_num['word_number'] = sentence_num['word_number'].astype('int')

In [44]:
sentence_num

,doc_id,sentence,word_number
0,T890221.159,67714.0,3
1,T890221.159,67715.0,37
2,T890221.159,67716.0,19
3,T890221.159,67717.0,22
4,T890221.159,67718.0,16
5,T890221.159,67719.0,37
6,T890221.159,67720.0,22
7,T890221.159,67721.0,31
8,T890221.159,67722.0,30
9,T890221.159,67723.0,18


Second: use the chain generated last step to generate a new dataframe with markables and features in it

In [ ]:
list_with_features = list()
i=0
for seg in groupby_doc_id['doc_id']:
    #find all markables in one segment
    segchain = chain_sort.loc[chain_sort['doc_id']==seg]
    #for each markable in the segment
    for j in range(len(segchain)):
        doc_id = segchain.iloc[j]['doc_id']
        sent_id = segchain.iloc[j]['sentence']
        start_token = segchain.iloc[j]['strat_token']
        end_token = segchain.iloc[j]['end_token']
        #find record in train_coref file to extract features
        index = df_corf[(df_corf['doc_id'] == doc_id) & (df_corf['sentence'] == int(sent_id)) & (df_corf['word_number'] == int(start_token))].index.tolist()
        POS = ""
        gramFct = ""
        FEATS = ""
        head_num = float("inf")
        #find the head for a mention with multiple words in it.
        if int(start_token) != int(end_token) :
            dis = int(end_token) - int(start_token)+1
            k=0
            for k in range(dis):
                head = int(df_sync_2.iloc[index[0]+k]['head'])
                if head <= head_num:
                    head_num = head
                    POS = df_sync_2.iloc[index[0]+k]['POS']
                    gramFct = df_sync_2.iloc[index[0]+k]['gramFct']
                    FEATS = df_sync_2.iloc[index[0]+k]['FEATS']
        #if there is only one word in the markable.
        else:
            POS = df_sync_2.iloc[index[0]]['POS']
            gramFct = df_sync_2.iloc[index[0]]['gramFct']
            FEATS = df_sync_2.iloc[index[0]]['FEATS']
            head_num = int(df_sync_2.iloc[index[0]]['head'])
        list_with_features.append([seg,sent_id,start_token,end_token,gramFct,POS,head_num,FEATS])
        i=i+1

In [48]:
list_with_features

[['T890221.159', 67714.0, 1.0, 1.0, 'nsubj', 'NE', 2, 'Case=Nom|Number=Sing'],
 ['T890221.159', 67714.0, 1.0, 1.0, 'nsubj', 'NE', 2, 'Case=Nom|Number=Sing'],
 ['T890221.159',
  67714.0,
  3.0,
  3.0,
  'obj',
  'NN',
  2,
  'Case=Acc|Gender=Fem|Number=Sing'],
 ['T890221.159', 67715.0, 4.0, 5.0, 'nmod', 'NE', 2, 'Case=Dat|Number=Sing'],
 ['T890221.159',
  67715.0,
  10.0,
  12.0,
  'nmod',
  'NN',
  7,
  'Case=Dat|Gender=Fem|Number=Sing'],
 ['T890221.159',
  67715.0,
  1.0,
  2.0,
  'det',
  'ART',
  2,
  'Case=Nom|Definite=Def|Gender=Masc|Number=Sing|PronType=Art'],
 ['T890221.159', 67715.0, 4.0, 5.0, 'nmod', 'NE', 2, 'Case=Dat|Number=Sing'],
 ['T890221.159',
  67715.0,
  7.0,
  7.0,
  'nmod',
  'NN',
  2,
  'Case=Acc|Gender=Fem|Number=Plur'],
 ['T890221.159',
  67715.0,
  9.0,
  10.0,
  'amod',
  'ADJA',
  12,
  'Case=Dat|Gender=Fem|Number=Sing'],
 ['T890221.159',
  67715.0,
  12.0,
  12.0,
  'nmod',
  'NN',
  7,
  'Case=Dat|Gender=Fem|Number=Sing'],
 ['T890221.159',
  67715.0,
  15.0

In [49]:
#change columns' names and data types for further use
markables_with_features = pd.DataFrame(list_with_features, columns=['seg','sent_id','start_token','end_token','gramFct','POS','head','FEATS'])
markables_with_features['sent_id'] = markables_with_features['sent_id'].astype('int')
markables_with_features['start_token'] = markables_with_features['start_token'].astype('int')
markables_with_features['end_token'] = markables_with_features['end_token'].astype('int')

In [53]:
#remove all rows with float("inf") in them
markables_with_features = markables_with_features[~markables_with_features['head'].isin([float("inf")])]

In [54]:
markables_with_features = markables_with_features.reset_index(drop=True)

In [55]:
markables_with_features['head'] = markables_with_features['head'].astype('int')

In [56]:
markables_with_features

,seg,sent_id,start_token,end_token,gramFct,POS,head,FEATS
0,T890221.159,67714,1,1,nsubj,NE,2,Case=Nom|Number=Sing
1,T890221.159,67714,1,1,nsubj,NE,2,Case=Nom|Number=Sing
2,T890221.159,67714,3,3,obj,NN,2,Case=Acc|Gender=Fem|Number=Sing
3,T890221.159,67715,4,5,nmod,NE,2,Case=Dat|Number=Sing
4,T890221.159,67715,10,12,nmod,NN,7,Case=Dat|Gender=Fem|Number=Sing
5,T890221.159,67715,1,2,det,ART,2,Case=Nom|Definite=Def|Gender=Masc|Number=Sing|...
6,T890221.159,67715,4,5,nmod,NE,2,Case=Dat|Number=Sing
7,T890221.159,67715,7,7,nmod,NN,2,Case=Acc|Gender=Fem|Number=Plur
8,T890221.159,67715,9,10,amod,ADJA,12,Case=Dat|Gender=Fem|Number=Sing
9,T890221.159,67715,12,12,nmod,NN,7,Case=Dat|Gender=Fem|Number=Sing


In [57]:
#save markables_with_features.csv
markables_with_features.to_csv(path_or_buf='markables_with_features.csv', sep=',',index = True, header = True,encoding='UTF-8')

In [59]:
#change data type for further use
df_golden_list['sent_id'] = df_golden_list['sent_id'].astype('float')
df_golden_list['start_token'] = df_golden_list['start_token'].astype('float')
df_golden_list['end_token'] = df_golden_list['end_token'].astype('float')
df_golden_list['head'] = df_golden_list['head'].astype('float')

In [60]:
df_golden_list['sent_id'] = df_golden_list['sent_id'].astype('int')
df_golden_list['start_token'] = df_golden_list['start_token'].astype('int')
df_golden_list['end_token'] = df_golden_list['end_token'].astype('int')
df_golden_list['head'] = df_golden_list['head'].astype('int')

Third: generate positive samples and negative samples based on true label

If two markables exist in the goldchain, if they have same coref_id then the pair formed by these two markables will be labeled as positive, if they don't share same coref_id then the pair formed by these two markables will be labeled as negative.

In [ ]:
gold_std = list()
gold_std_neg = list()
for seg in groupby_doc_id['doc_id']:
    #find all markables in one segment
    markable_all = markables_with_features.loc[chain_sort['doc_id']==seg]
    #find all mentions in one segment
    gold_all = df_golden_list.loc[df_golden_list['seg']==seg]
    #generate pairs
    for i in range(len(markable_all)-1):  
        for j in range((i+1),len(markable_all)):
            #some features
            sent_i = int(markable_all.iloc[i]['sent_id'])
            start_token_i = markable_all.iloc[i]['start_token']
            end_token_i = markable_all.iloc[i]['end_token']
            gramFct_i = markable_all.iloc[i]['gramFct']
            POS_i = markable_all.iloc[i]['POS']
            head_i = markable_all.iloc[i]['head']
            FEATS_i = markable_all.iloc[i]['FEATS']
            #extract Case, Gender and Number info from FEATS
            if 'Case' in FEATS_i.split('|')[0]:
                Case_i = FEATS_i.split('|')[0].split('=')[1]
            else: Case_i = 0
            for feat in FEATS_i.split('|'):
                if 'Gender' in feat:
                    Gender_i = feat.split('=')[1]
                    break
                else: Gender_i=0
            for feat in FEATS_i.split('|'):
                if 'Number' in feat:
                    Number_i = feat.split('=')[1]
                    break
                else: Number_i=0
            
            
            #some features
            sent_j = int(markable_all.iloc[j]['sent_id'])
            start_token_j = markable_all.iloc[j]['start_token']
            end_token_j = markable_all.iloc[j]['end_token']
            gramFct_j = markable_all.iloc[j]['gramFct']
            POS_j = markable_all.iloc[j]['POS']
            head_j = markable_all.iloc[j]['head']
            FEATS_j = markable_all.iloc[j]['FEATS']
            #extract Case, Gender and Number info from FEATS
            if 'Case' in FEATS_j.split('|')[0]:
                Case_j = FEATS_j.split('|')[0].split('=')[1] 
            else : Case_j = 0
            for feat in FEATS_j.split('|'):
                if 'Gender' in feat:
                    Gender_j = feat.split('=')[1]
                    break
                else: Gender_j=0
            for feat in FEATS_j.split('|'):
                if 'Number' in feat:
                    Number_j = feat.split('=')[1]
                    break
                else: Number_j=0
            
            #window-based strategy, window size:2
            if abs(sent_i-sent_j)<=2:
                #look up for two markables in goldchain
                gold_i = gold_all.loc[(gold_all['sent_id']==sent_i)&(gold_all['start_token']==start_token_i)&(gold_all['end_token']==end_token_i)&(gold_all['head']==head_i)&(gold_all['POS']==POS_i)&(gold_all['FEATS']==FEATS_i)&(gold_all['gramFct']==gramFct_i)]
                gold_j = gold_all.loc[(gold_all['sent_id']==sent_j)&(gold_all['start_token']==start_token_j)&(gold_all['end_token']==end_token_j)&(gold_all['head']==head_j)&(gold_all['POS']==POS_j)&(gold_all['FEATS']==FEATS_j)&(gold_all['gramFct']==gramFct_j)]
                #if they exist in goldchain
                if gold_i.empty == False and gold_j.empty == False:
                    corf_id_i = int(gold_i['core_id'])
                    corf_id_j = int(gold_j['core_id'])
                    #if they share same coref_id, then this pair labeles as positive
                    if corf_id_i == corf_id_j:
                        #some features
                        if POS_i == POS_j:
                            samePOS = 1
                        else : samePOS = 0
                        if head_i == head_j:
                            sameHead = 1
                        else : sameHead = 0
                        if gramFct_i == gramFct_j:
                            sameGramFct = 1
                        else : sameGramFct = 0
                        if FEATS_i == FEATS_j:
                            matchOrNot = 1
                        else : matchOrNot = 0
                        if Case_i == Case_j:
                            sameCase = 1
                        else : sameCase = 0
                        if Gender_i == Gender_j == 0:
                            sameGender = 1
                        elif Gender_i == Gender_j:
                            sameGender = 1
                        else : sameGender = 0
                        if Number_i == Number_j == 0:
                            sameNumber = 1
                        elif Number_i == Number_j:
                            sameNumber = 1
                        else : sameNumber = 0
                        label = 1
                        
                        #calculate distance
                        s_distance = 0
                        w_place_1 = 0
                        w_place_2 = 0
                        w_distance = 0
                        distance = abs(int(sent_i)-int(sent_j))
                        
                        #if these two markables are not in the same or neighbor sentences
                        if distance >1:
                            #add all words between the two sentences where these two markables' head exist
                            for k in range(1,distance):
                                if sentence_num.loc[sentence_num['sentence']==(int(sent_i)+k)]['word_number'].empty:
                                    s_distance = s_distance+0
                                else:
                                    s_distance = s_distance + int(sentence_num.loc[sentence_num['sentence']==(int(sent_i)+k)]['word_number'].tolist()[0])
                            #add the words before the second markable's head and behind the first markable's head
                            w_place_1 = int(sentence_num.loc[sentence_num['sentence']==int(sent_j)]['word_number'])-int(head_j)
                            w_place_2 = int(head_i)-1
                            w_distance = w_distance + s_distance + w_place_1 + w_place_2
                            
                        #if these two markables are in neighbor sentences
                        elif distance ==1:
                            #add the words before the second markable's head and behind the first markable's head
                            w_place_1 = int(sentence_num.loc[sentence_num['sentence']==int(sent_j)]['word_number'])-int(head_j)
                            w_place_2 = int(head_i)-1
                            w_distance = w_distance + s_distance + w_place_1 + w_place_2
                            
                        #if these two markables are in same sentences
                        elif distance ==0:
                            #add words between heads of two markables
                            w_place_1 = int(sentence_num.loc[sentence_num['sentence']==int(sent_j)]['word_number'])-int(head_j)
                            w_place_2 = int(head_i)
                            if w_place_1 == w_place_2:
                                w_distance = w_distance + s_distance + abs(w_place_1 - w_place_2)
                            else : w_distance = w_distance + s_distance + abs(w_place_1 - w_place_2)-1
                        
                        #append the record to positive list
                        gold_std.append([label,w_distance,POS_i,gramFct_i,Case_i,Gender_i,Number_i,POS_j,gramFct_j,Case_i,Gender_j,Number_j,sameHead,samePOS,sameGramFct,matchOrNot,sameCase,sameGender,sameNumber])
                    
                    #else if they do not have same cored_id ,then this pair labeles as negative
                    else:
                        #some features
                        if POS_i == POS_j:
                            samePOS_n = 1
                        else : samePOS_n = 0
                        if head_i == head_j:
                            sameHead_n = 1
                        else : sameHead_n = 0
                        if gramFct_i == gramFct_j:
                            sameGramFct_n = 1
                        else : sameGramFct_n = 0
                        if FEATS_i == FEATS_j:
                            matchOrNot_n = 1
                        else : matchOrNot_n = 0
                        if Case_i == Case_j:
                            sameCase_n = 1
                        else : sameCase_n = 0
                        if Gender_i == Gender_j == 0:
                            sameGender_n = 1
                        elif Gender_i == Gender_j:
                            sameGender_n = 1
                        else : sameGender_n = 0
                        if Number_i == Number_j == 0:
                            sameNumber_n = 1
                        elif Number_i == Number_j:
                            sameNumber_n = 1
                        else : sameNumber_n = 0
                        label_n = 0
                        
                        #calculate distance, same steps as for positive records
                        n_s_distance = 0
                        n_w_place_1 = 0
                        n_w_place_2 = 0
                        n_w_distance = 0
                        n_distance = abs(int(sent_i)-int(sent_j))
                        if n_distance >1:
                            for k in range(1,n_distance):
                                if sentence_num.loc[sentence_num['sentence']==(int(sent_i)+k)]['word_number'].empty: 
                                            n_s_distance = n_s_distance+0
                                else:
                                    n_s_distance = n_s_distance + int(sentence_num.loc[sentence_num['sentence']==(int(sent_i)+k)]['word_number'].tolist()[0])
                            n_w_place_1 = int(sentence_num.loc[sentence_num['sentence']==int(sent_j)]['word_number'])-int(head_j)
                            n_w_place_2 = int(head_i)-1
                            n_w_distance = n_w_distance + n_s_distance + n_w_place_1 + n_w_place_2
                        elif n_distance ==1:
                            n_w_place_1 = int(sentence_num.loc[sentence_num['sentence']==int(sent_j)]['word_number'])-int(head_j)
                            n_w_place_2 = int(head_i)-1
                            n_w_distance = n_w_distance + n_s_distance + n_w_place_1 + n_w_place_2
                        elif n_distance ==0:
                            n_w_place_1 = int(sentence_num.loc[sentence_num['sentence']==int(sent_j)]['word_number'])-int(head_j)
                            n_w_place_2 = int(head_i)
                            if n_w_place_1 == n_w_place_2:
                                n_w_distance = n_w_distance + n_s_distance + abs(n_w_place_1 - n_w_place_2)
                            else : 
                                n_w_distance = n_w_distance + n_s_distance + abs(n_w_place_1 - n_w_place_2)-1
                                
                        #append the record to negative list    
                        gold_std_neg.append([label_n,n_w_distance,POS_i,gramFct_i,Case_i,Gender_i,Number_i,POS_j,gramFct_j,Case_j,Gender_j,Number_j,sameHead_n,samePOS_n,sameGramFct_n,matchOrNot_n,sameCase_n,sameGender_n,sameNumber_n])
                else:
                    break
                    
            else:
                break
    

In [64]:
#convert list to dataframe for both negative list and positive list
df_gold_std = pd.DataFrame(gold_std, columns=['label','distance','POS1','gramFct1','Case1','Gender1','Number1','POS2','gramFct2','Case2','Gender2','Number2','sameHead','samePOS','sameGramFct','matchOrNot','sameCase','sameGender','sameNumber'])
df_neg_gold_std = pd.DataFrame(gold_std_neg, columns=['label','distance','POS1','gramFct1','Case1','Gender1','Number1','POS2','gramFct2','Case2','Gender2','Number2','sameHead','samePOS','sameGramFct','matchOrNot','sameCase','sameGender','sameNumber'])

In [65]:
#append negative samples and positive samples together form a new dataframe which contains all samples
df_gold_std_all = df_gold_std.append(df_neg_gold_std)

In [66]:
#save test file
df_gold_std_all.to_csv(path_or_buf='test_data_with_case_and_gender_number.csv', sep=',',index = True, header = True,encoding='UTF-8')

In [265]:
df_gold_std_all

,label,distance,POS1,gramFct1,Case1,Gender1,POS2,gramFct2,Case2,Gender2,sameHead,samePOS,sameGramFct,matchOrNot,sameCase,sameGender
0,1,0,NE,nsubj,Nom,0,NE,nsubj,Nom,0,1,1,1,1,1,1
1,1,4,NE,nsubj,Nom,Fem,PPER,nsubj:pass,Nom,Fem,0,0,0,0,1,1
2,1,21,NE,nsubj,Nom,Fem,PPER,nsubj,Nom,Fem,0,0,1,0,1,1
3,1,18,NE,flat,Dat,0,NE,flat,Dat,0,0,1,1,0,0,1
4,1,3,NN,nsubj,Nom,Neut,PRELS,nsubj,Nom,0,0,0,1,0,1,0
5,1,32,NE,flat,Nom,0,NE,flat,Nom,0,1,1,1,1,1,1
6,1,6,ART,det,Nom,Fem,PRELS,obj,Nom,0,0,0,0,0,0,0
7,1,7,PPER,obj,Dat,0,PPER,nsubj,Dat,0,0,1,0,0,0,1
8,1,0,NE,nmod,Acc,Masc,PRELS,obj,Acc,Masc,0,0,0,0,1,1
9,1,10,ART,det,Acc,Neut,PRELS,obj,Acc,Neut,0,0,0,0,1,1
